# 1. Mount Azure Data Lake using Service Principal
#### Steps to follow
1. Get client_id, tenant_id and client_secret from key vault
2. Set Spark Config with App/ Client Id, Directory/ Tenant Id & Secret
3. Call file system utlity mount to mount the storage
4. Explore other file system utlities related to mount (list all mounts, unmount)

Obtenemos las credenciales necesarias para montar un almacenamiento en Microsoft Data Lake

In [0]:
#Get client_id, tenant_id and client_secret from key vault
client_id = dbutils.secrets.get(scope = 'mlops-scope', key = 'mlops-app-client-id')
tenant_id = dbutils.secrets.get(scope = 'mlops-scope', key = 'mlops-app-tenant-id')
client_secret = dbutils.secrets.get(scope = 'mlops-scope', key = 'mlops-app-client-secret')

Asignamos las variables necesariamos para montar algun Data Lake

In [0]:
#Set Spark Config with App/ Client Id, Directory/ Tenant Id & Secret
configs = {"fs.azure.account.auth.type": "OAuth",
          "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
          "fs.azure.account.oauth2.client.id": client_id,
          "fs.azure.account.oauth2.client.secret": client_secret,
          "fs.azure.account.oauth2.client.endpoint": f"https://login.microsoftonline.com/{tenant_id}/oauth2/token"}

Montamos el almacenamiento de Data Lake que deseamos

In [0]:
#Call file system utlity mount to mount the storage
dbutils.fs.mount(
 source = "abfss://presentation@datalakemlopsd4m.dfs.core.windows.net/",
 mount_point = "/mnt/datalakemlopsd4m/presentation/",
 extra_configs = configs)

In [0]:
display(dbutils.fs.ls("/"))

In [0]:
#Verificar que contenedores estan montados en el azure datalake
#%fs mounts

#Desmontar el Contenedor especifico
#dbutils.fs.unmount("/mnt/datalakemlopsd4m/demo/")

#Ver cuales son los archivos que estan en el Directorio
#display(dbutils.fs.ls("/mnt/datalakemlopsd4m/raw/marcobre/turno1/"))

#Revisar el el bd cargado apartir de spark
#display(spark.read.csv("/mnt/datalakemlopsd4m/raw/marcobre/turno1/datos_turno1_vf.csv", header=True))

# **2. Comprensión de los Datos(EDA) y Preprocesamiento de Datos**

### *2.1 Cargamos los datos desde el Storage (RAW) para realizar la comprension y preparacion de datos*

In [0]:
import pandas as pd
# Cargar el archivo CSV en un DataFrame de Spark
datos_turno1 = spark.read.csv("/mnt/datalakemlopsd4m/raw/marcobre/turno1/datos_turno1_vf.csv", header=True)
datos_turno2 = spark.read.csv("/mnt/datalakemlopsd4m/raw/marcobre/turno1/datos_turno1_vf.csv", header=True)

# Convertir DataFrame de Spark a DataFrame de Pandas
df_turno1 = datos_turno1.toPandas()
df_turno2 = datos_turno2.toPandas()

# Agregar columna "turno" con valor 1 y 2 al DataFrame df_turno1
df_turno1['turno'] = 1
df_turno2['turno'] = 2

#Consolidar los datos del turno 1 y turno 2
datos = pd.concat([df_turno1, df_turno2], ignore_index=True)

# Ahora puedes manejar el DataFrame con Pandas
# Por ejemplo, puedes usar funciones de Pandas como head(), describe(), etc.
datos.head()

#### Paso Final (el DF final de pandas, debes convertirlo a un DF de SPARK, para que los datos limipios sean reflejados)

1. Convertimos el df-pandas a un df-spark, para poder guardarlo en el Data Storage

In [0]:
#Guardar luego de convertirlo a un DataFrame de Spark
spark_datos = spark.createDataFrame(datos)

2. Convertimos el tipo de datos Void a String, para poder guardar en un archivo CSV (Data Storage)

In [0]:
from pyspark.sql.functions import when, col

# Lista de todas las columnas
columnas = spark_datos.columns

# Convertir valores 'void' a cadena vacía ('') en todas las columnas que contienen 'void'
for columna in columnas:
    spark_datos = spark_datos.withColumn(
        columna,
        when(col(columna) == 'void', '').otherwise(col(columna))
    )

3. Guardamos el df-spark en la ruta del Data Storage de Microsoft Azure

In [0]:
# Ruta donde quieres guardar el archivo CSV en tu Azure Data Lake Storage
ruta_guardado = "/mnt/datalakemlopsd4m/raw/marcobre/datosraw/datostotalmarcobre.csv"

# Guardar el DataFrame en formato CSV en la ruta especificada
spark_datos.write.csv(ruta_guardado, header=True, mode="overwrite")
#spark_datos.repartition(1).write.csv(ruta_guardado, header=True, mode="overwrite") #Sirve para guardar solo en 1 partition csv

4. Cargamos el CSV(Carpeta Spark) del Data Storage, para verificar que todo este OK (Tambien se convierte de dfSpark a dfPandas)

In [0]:
# Lee el archivo CSV en un DataFrame de Spark
ruta_carpeta_csv = "/mnt/datalakemlopsd4m/raw/marcobre/datosraw/datostotalmarcobre.csv"
df_spark = spark.read.option("header", "true").csv(ruta_carpeta_csv)

# Muestra los primeros registros del DataFrame de Spark Convertido a un DataFrame Pandas
df_pandas = df_spark.toPandas()
df_pandas.head(3)

In [0]:
df_pandas.shape

In [0]:
datos = df_pandas

In [0]:
import pandas as pd
#Configuramos pandas para que podamos vizualizar todas las columnas y filas la estadistica descriptiva de todas las variables
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)
#Configuramos pandas para que lanze valores con una precision de hasta 6 decimales
pd.set_option('display.float_format', '{:.6f}'.format)

In [0]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# 3.Tratamiento de valores Nulos
# 3.1 Supongamos que tienes un DataFrame llamado datos
valores_nulos = datos.isnull().sum()
valores_nulos_ordenados = valores_nulos.sort_values(ascending=False)
porcentaje_nulos = (valores_nulos_ordenados / len(datos)) * 100

In [0]:
columnas_a_eliminar = porcentaje_nulos[porcentaje_nulos > 80].index
datos = datos.drop(columnas_a_eliminar, axis=1)

In [0]:
datos = datos.drop(['tipoubicacionsupervisor_camion','tipoubicacionsupervisor_pala','id_cargadescarga_pases','dumpreal','loadreal', 'rownum_global'], axis=1)

In [0]:
datos.isnull().sum()

In [0]:
# Calcula la moda de 'has_block_pases'
moda_has_block_pases = datos['has_block_pases'].mode()[0]

# Completa los valores nulos con la moda en la columna 'has_block_pases'
datos['has_block_pases'].fillna(moda_has_block_pases, inplace=True)

In [0]:
moda_tipodescargaidentifier = datos['tipodescargaidentifier'].mode()[0]

# Completa los valores nulos con la moda en la columna 'tipodescargaidentifier'
datos['tipodescargaidentifier'].fillna(moda_tipodescargaidentifier, inplace=True)

Completa los datos Nulos por 0

In [0]:
datos = datos.fillna(0)

In [0]:
datos = datos.drop_duplicates()
datos.shape

In [0]:
datos.dtypes

In [0]:
# Supongamos que 'datos' es tu DataFrame y 'columnas_fecha' son las columnas que contienen fechas
columnas_fecha = ['tiem_llegada_global', 'tiem_esperando', 'tiem_cuadra', 'tiem_cuadrado', 'tiem_carga', 'tiem_acarreo', 'tiem_cola', 'tiem_retro', 'tiem_listo', 'tiem_descarga', 'tiem_viajando', 'tiempo_inicio_carga_carguio', 'tiempo_esperando_carguio', 'previous_esperando_pala', 'tiempo_inicio_cambio_estado_camion', 'tiempo_inicio_cambio_estado_pala']

# Convertir todas las fechas al mismo formato
for columna in columnas_fecha:
    datos[columna] = pd.to_datetime(datos[columna], errors='coerce') #Si hubiese una fecha con  Error, lo reemplaza con NAT

In [0]:
# Reemplazar los valores nulos con la fecha de la fila anterior más 3 segundos adicionales
for columna in columnas_fecha:
    mask_nat = datos[columna].isna()
    datos[columna].loc[mask_nat] = datos[columna].fillna(method='ffill') + pd.to_timedelta(3, unit='s')

In [0]:
# Formatear las fechas en el formato deseado
formato_deseado = "%Y-%m-%d %H:%M:%S.%f%z"  # Formato deseado
for columna in columnas_fecha:
    datos[columna] = datos[columna].dt.strftime(formato_deseado)
    datos[columna] = pd.to_datetime(datos[columna])
    datos[columna] = datos[columna] + pd.to_timedelta(999, unit='ms')

In [0]:
import pandas as pd

# Supongamos que 'datos' es tu DataFrame

# Diccionario para especificar los tipos de datos deseados para cada columna
tipos_de_datos = {
    'id_ciclo_acarreo': 'int64','id_cargadescarga': 'int64','id_palas': 'int64','id_equipo_camion': 'int64','id_ciclo_carguio': 'float64',
    'id_equipo_carguio': 'float64','id_trabajador_pala': 'float64','id_guardia_realiza_carga_al_camion': 'float64','id_locacion': 'float64',
    'id_poligono_se_obtiene_material': 'float64','tiempo_ready_cargando_pala': 'float64','tiempo_ready_esperando_pala': 'float64',
    'cantidad_equipos_espera_al_termino_carga_pala': 'float64','id_estados_camion': 'int64','id_equipo_table_estados_camion': 'int64',
    'id_detal_estado_camion': 'int64','tiempo_estimado_duracion_estado_camion': 'int64','en_campo_o_taller_mantenimiento_camion': 'int64',
    'id_tipo_estad_camion': 'int64','id_estados_pala': 'float64','id_equipo_table_estados_pala': 'float64','id_detal_estado_pala': 'float64',
    'tiempo_estimado_duracion_estado_pala': 'float64','en_campo_o_taller_mantenimiento_pala': 'float64','id_tipo_estad_pala': 'float64',
    'id_descarga': 'int64','id_factor': 'int64','id_poligono': 'float64','tiempo_ready_llegada_esperando': 'float64',
    'tiempo_ready_esperando_cuadra': 'float64','tiempo_ready_cuadra_cuadrado': 'float64', 'tiempo_ready_cuadrado_cargado': 'float64',
    'tiempo_ready_carga_acarreo': 'float64','tiempo_ready_acarreo_cola': 'float64','tiempo_ready_cola_retro': 'float64',
    'tiempo_ready_retro_listo': 'float64','tiempo_ready_listo_descarga': 'float64','tiempo_ready_descarga_viajandovacio': 'float64',
    'id_trabajador_camion': 'int64','id_palanext': 'int64','tonelaje': 'float64','tonelajevims': 'float64','yn_estado': 'bool',
    'id_guardia_hizocarga': 'int64','id_guardia_hizodescarga': 'int64','id_zona_aplicafactor': 'int64','id_zona_pertenece_poligono': 'float64',
    'factor': 'int64','toneladas_secas': 'float64','productividad_operativa_acarreo_tn_h': 'float64','productividad_operativa_carguio_tn_h': 'float64','efhcargado': 'float64','efhvacio': 'float64','distrealcargado': 'float64','distrealvacio': 'float64','coorxdesc': 'float64',
    'coorydesc': 'float64','coorzdesc': 'float64','tipodescargaidentifier': 'float64','tonelajevvanterior': 'int64','tonelajevvposterior': 'float64','velocidadvimscargado': 'float64','velocidadvimsvacio': 'float64','velocidadgpscargado': 'float64','velocidadgpsvacio': 'float64','tonelajevimsretain': 'float64', 'nivelcombuscargado': 'float64','nivelcombusdescargado':'float64',
    'volumen': 'float64','aplicafactor_vol': 'bool','coorzniveldescarga': 'float64','efh_factor_loaded': 'float64','efh_factor_empty':'float64',
    'id_secundario': 'int64','id_principal': 'int64','capacidad_vol_equipo': 'float64','capacidad_pes_equipo': 'float64',
    'capacidadtanque_equipo': 'int64','peso_bruto_equipo': 'float64','ishp_equipo': 'bool','ancho_equipo': 'int64','largo_equipo': 'int64',
    'numeroejes_equipo': 'int64','id_turnos_turnocarga': 'int64','horaini_turnocarga': 'int64','horafin_turnocarga': 'int64',
    'id_turnos_turnodescarga': 'int64','horaini_turnodescarga': 'int64','horafin_turnodescarga': 'int64',
    'id_zona_encuentra_descarga': 'int64','id_nodo_carga': 'float64','id_nodo_descarga': 'int64',
    'elevacion_descarga': 'int64','nivel_elevacion_locacion_mts': 'float64','radio_locacion': 'float64','id_material': 'float64',
    'elevacion_poligono_mts': 'float64','densidad_poligono': 'float64','tonelaje_inicial_poligono': 'float64','id_pases': 'float64',
    'id_palas_pases': 'float64','angulo_giro_promedio_pases': 'float64','has_block_pases': 'bool'}

# Convertir las columnas al tipo de dato correspondiente
for columna, tipo in tipos_de_datos.items():
    datos[columna] = datos[columna].astype(tipo)

In [0]:
#for columna, tipo in tipos_de_datos.items():
#    if columna in datos.columns:
#        datos[columna] = datos[columna].astype(tipo)
#    else:
#        print(f"La columna '{columna}' no está presente en el DataFrame.")

Transformacion de datos que tienen medidas distintas (ejemplo: cm, metros, km, etc)

In [0]:
#pasar tonelajevims a toneladas
datos['tonelajevims'] = datos['tonelajevims'] / 10

#pasar efhcargado de centimetros a metros
datos['efhcargado'] = datos['efhcargado'] / 100

#pasar efhvacio de centimetros a metros
datos['efhvacio'] = datos['efhvacio'] / 100

#pasar distrealcargado de centimetros a metros
datos['distrealcargado'] = datos['distrealcargado'] / 100

#pasar distrealvacio de centimetros a metros
datos['distrealvacio'] = datos['distrealvacio'] / 100

#pasar coorxdesc de centimetros a Km
datos['coorxdesc'] = datos['coorxdesc'] / 100000

#pasar coorydesc de centimetros a Km
datos['coorydesc'] = datos['coorydesc'] / 100000

#pasar coorzdesc de centimetros a Km
datos['coorzdesc'] = datos['coorzdesc'] / 100000

#pasar velocidadvimscargado a  Km/hr
datos['velocidadvimscargado'] = datos['velocidadvimscargado'] / 1000

#pasar velocidadvimsvacio a  Km/hr
datos['velocidadvimsvacio'] = datos['velocidadvimsvacio'] / 1000

#pasar velocidadvimsvacio a  Km/hr
datos['velocidadgpscargado'] = datos['velocidadgpscargado'] / 1000

#pasar velocidadvimsvacio a  Km/hr
datos['velocidadgpsvacio'] = datos['velocidadgpsvacio'] / 1000

#pasar tonelajevimsretain a  toneladas
datos['tonelajevimsretain'] = datos['tonelajevimsretain'] / 10

#pasar coorzniveldescarga de centimetros a metros
datos['coorzniveldescarga'] = datos['coorzniveldescarga'] / 100

#pasar ancho_equipo de centimetros a metros
datos['ancho_equipo'] = datos['ancho_equipo'] / 100

#pasar largo_equipo de centimetros a metros
datos['largo_equipo'] = datos['largo_equipo'] / 100

#pasar tonelajevvanterior a toneladas
datos['tonelajevvanterior'] = datos['tonelajevvanterior'] / 10

#pasar tonelajevvposterior a toneladas
datos['tonelajevvposterior'] = datos['tonelajevvposterior'] / 10

In [0]:
datos['termino_carga_equipo_en_espera_cuadrado_cuadrandose_carguio'].value_counts()

In [0]:
datos['termino_carga_equipo_en_espera_cuadrado_cuadrandose_carguio'].unique()

In [0]:
datos['termino_carga_equipo_en_espera_cuadrado_cuadrandose_carguio'].dtypes

In [0]:
datos['termino_carga_equipo_en_espera_cuadrado_cuadrandose_carguio'].mode().iloc[0]

In [0]:
# Reemplazar '0' por la moda
datos['termino_carga_equipo_en_espera_cuadrado_cuadrandose_carguio'] = datos['termino_carga_equipo_en_espera_cuadrado_cuadrandose_carguio'].replace(0,  datos['termino_carga_equipo_en_espera_cuadrado_cuadrandose_carguio'].mode().iloc[0])

# Reemplazar 'True' por True y 'False' por False , Paso crucial para antes de Convertir a DATOS BOOLEANO
datos['termino_carga_equipo_en_espera_cuadrado_cuadrandose_carguio'] = datos['termino_carga_equipo_en_espera_cuadrado_cuadrandose_carguio'].replace({'True': True, 'False': False})

# Convertir la columna a tipo de datos booleano
datos['termino_carga_equipo_en_espera_cuadrado_cuadrandose_carguio'] = datos['termino_carga_equipo_en_espera_cuadrado_cuadrandose_carguio'].astype(bool)

In [0]:
# Reemplazar '0' por la moda
datos['cambio_estado_operatividad_carguio'] =datos['cambio_estado_operatividad_carguio'].replace(0, datos['cambio_estado_operatividad_carguio'].mode().iloc[0])

# Reemplazar 'True' por True y 'False' por False , Paso crucial para antes de Convertir a DATOS BOOLEANO
datos['cambio_estado_operatividad_carguio'] = datos['cambio_estado_operatividad_carguio'].replace({'True': True, 'False': False})

# Convertir la columna a tipo de datos booleano
datos['cambio_estado_operatividad_carguio'] = datos['cambio_estado_operatividad_carguio'].astype(bool)

In [0]:
# Verificar si la variable es True
if datos['cambio_estado_operatividad_carguio'].iloc[16] == True:  #  datos['cambio_estado_operatividad_carguio'].iloc[32] == True
    print("La variable es True")
else:
    print("La variable no es True")

In [0]:
datos.dtypes

In [0]:
import pandas as pd

# Define un diccionario con los nuevos nombres de las columnas solo para algunas columnas
nuevos_nombres = {'id_cargadescarga' : 'id_cargadescarga_ciclo',
    'termino_carga_equipo_en_espera_cuadrado_cuadrandose_carguio' : 'al_termino_cargar_en_espera_cuadrado_cuadrandose',
    'id_descarga' : 'id_zona_hace_descarga',  
    'tiem_llegada_global': 'tiempo_llegada_camion', 'tiem_esperando': 'tiempo_esperando_camion_en_locacion', 
    'tiem_cuadra': 'tiempo_cuadra_camion','tiem_cuadrado': 'tiempo_cuadrado_camion' , 'tiem_carga' : 'tiempo_cargar_al_camion', 
    'tiem_acarreo' : 'tiempo_acarreo_camion', 'tiem_cola': 'tiempo_cola_camion_en_zonadescarga','tiem_retro': 'tiempo_retroceso_para_descargar',
    'tiem_listo' : 'tiempo_listo_para_descargar',  'tiem_descarga': 'tiempo_descarga_camion', 'tiem_viajando': 'tiempo_viajando_vacio_locacion', 
    'tonelaje':'tonelaje_nominal', 'tonelajevims':'tonelaje_segun_computadora', 'yn_estado': 'cambios_estado_en_ciclo',
    'distrealcargado': 'distancia_recorrida_camioncargado_km_gps_mts', 'distrealvacio': 'distancia_recorrida_camionvacio_km_gps_mts' ,
    'coorxdesc': 'coordenada_x_descarga_km', 'coorydesc': 'coordenada_y_descarga_km' , 'coorzdesc': 'coordenada_z_descarga_km',
    'tipodescargaidentifier': 'tipo_descarga_efectuado', 
    'tonelajevvanterior': 'tonelaje_camion_viajevacio_cicloanterior_vims', 'tonelajevvposterior': 'tonelaje_camion_viajevacio_cicloactual_vims',
    'velocidadvimscargado': 'promedio_velocidad_camioncargado_km/hr_compu', 
    'velocidadvimsvacio': 'promedio_velocidad_camionvacio_km/hr_compu',
    'velocidadgpscargado':'promedio_velocidad_camioncargado_km/hr_gps', 'velocidadgpsvacio': 'promedio_velocidad_camionvacio_km/hr_gps', 
    'tonelajevimsretain': 'tonelaje_camion_antes_cargaestabilizada', 'nivelcombuscargado': 'porcentaje_combustible_camioncargando', 
    'nivelcombusdescargado':'porcentaje_combustible_camiondescargando', 'volumen': 'volumen_nominal', 'aplicafactor_vol': 'aplica_factor_volumen_o_tonelaje',
    'coorzniveldescarga': 'nivel_descarga_metros', 'nombre_equipo':'nombre_equipo_acarreo', 
    'id_secundario':'id_flota_secundaria', 'flota_secundaria':'nombre_flota_secundaria', 'id_principal': 'id_flota_principal', 'flota_principal':'nombre_flota_principal',
    'capacidad_vol_equipo': 'capacidad_en_volumen_equipo_acarreo_m3', 'capacidad_pes_equipo':'capacidad_en_peso_equipo_acarreo', 'capacidadtanque_equipo': 'capacidadtanque_equipoacarreo_galones',
    'peso_bruto_equipo':'peso_bruto_equipo_acarreo', 'ishp_equipo':'si_no_equipo_altaprecision', 'ancho_equipo':'ancho_equipo_metros', 'largo_equipo':'largo_equipo_metros',
    'elevacion_descarga':'nivel_elevacion_descarga_metros', 'nombre_descarga':'nombre_zona_descarga', 
    'nombre_carga_locacion':'nombre_locacion_carga', 'nivel_elevacion_locacion_mts':'nivel_elevacion_locacion_carga_metros', 'radio_locacion':'radio_locacion_metros',
    'ids_poligonos_en_locacion':'ids_poligonos_en_locacion_carga', 'id_material': 'id_material_dominante_en_poligono', 
    'elevacion_poligono_mts':'elevacion_poligono_metros', 'ley_in':'lista_leyes', 'densidad_poligono':'densidad_inicial_poligono_creado_tn/m3',
    'capacidad_vol' : 'capacidad_en_volumen_equipo_carguio_m3', 'capacidad_pes':'capacidad_en_peso_equipo_carguio', 'capacidadtanque': 'capacidadtanque_equipocarguio_galones',
    'radiohexagonocuchara' : 'radiohexagonocuchara_equipocarguio', 'id_tablegen' : 'id_guardia_acarreocarga', 'nombre_tablegen' : 'nombre_guardia_acarreocarga', 
    'id_guardiadescarga': 'id_guardia_acarreodescarga', 'nombre_guardiadescarga':'nombre_guardia_acarreodescarga',
    'id':'id_guardia_carguio', 'nombre': 'nombre_guardia_carguio', 'id_locacion' : 'id_locacion_hace_carga','tonelaje_inicial_poligono': 'tonelaje_inicial_poligono_creado',  'efhvacio':'efhvacio_mts', 'efhcargado':'efhcargado_mts', 
}
# Renombra las columnas del DataFrame
datos = datos.rename(columns=nuevos_nombres)

In [0]:
import numpy as np
# 11. Agregamos Nuevas variables calculadas
# Agregamos la variable 'porcentaje_eficiencia_toneladas_movidas_acarreo'
datos['porcentaje_eficiencia_toneladas_movidas_acarreo'] = (datos['tonelaje_segun_computadora'] / datos['tonelaje_nominal']) * 100

#Agregamos la variable 'altura_elevacion'
datos['altura_elevacion'] = abs(datos['nivel_elevacion_descarga_metros'] - datos['nivel_elevacion_locacion_carga_metros'] )

# Agregamos la variable 'factor_perfil_rutavacio_mts'
datos['factor_perfil_rutavacio'] = np.where(datos['distancia_recorrida_camionvacio_km_gps_mts'] != 0,
                                            datos['efhvacio_mts'] / datos['distancia_recorrida_camionvacio_km_gps_mts'],
                                            0)

# Agregamos la variable 'factor_perfil_rutacargado_mts'
datos['factor_perfil_rutacargado'] = np.where(datos['distancia_recorrida_camioncargado_km_gps_mts'] != 0,
                                              datos['efhcargado_mts'] / datos['distancia_recorrida_camioncargado_km_gps_mts'],
                                              0)

# Agregamos la variable calculada "numero_pases_carguio" basado en la columna 'coord_x_pases'
datos['numero_pases_carguio'] = datos['coord_x_pases'].apply(lambda x: len(eval(x)) if isinstance(x, str) and '[' in x else x if isinstance(x, int) else 0)

#Agregamos la variable Galones_diponible_camioncargando
datos['Galones_disponibles_camioncargando'] = (datos['porcentaje_combustible_camioncargando']/100) * datos['capacidadtanque_equipoacarreo_galones']
#datos['demanda_galones_camioncargando'] = (datos['porcentaje_combustible_camioncargando']/100) * datos['capacidadtanque_equipoacarreo_galones']

#Agregar la variable Galones_diponible_camiondescargando 
datos['Galones_disponibles_camiondescargando'] = (datos['porcentaje_combustible_camiondescargando']/100) * datos['capacidadtanque_equipoacarreo_galones']

#Agregar la variable Galones_consumidos_entre_cargando_descargando 
datos['Galones_consumidos_entre_cargando_descargando_acarreo'] = datos['Galones_disponibles_camioncargando'] - datos['Galones_disponibles_camiondescargando']

## **3. Guardar los Datos procesados(spark df) apartir del storage RAW, en una tabla DELTA (hacia storage PROCESSED)**

Montamos el almacenamiento Processed de Data Lake donde gaurdaremos estos datos si no esta

In [0]:
#Call file system utlity mount to mount the storage
dbutils.fs.mount(
  source = "abfss://processed@datalakemlopsd4m.dfs.core.windows.net/",
  mount_point = "/mnt/datalakemlopsd4m/processed/",
  extra_configs = configs)

Creamos la BD donde almacenaremos las Tables DELTA (Verificar si ya esta creada)

In [0]:
# Listar todas las bases de datos
spark.sql("SHOW DATABASES").show()

In [0]:
# Crear la base de datos si no existe en el almacenamiento de PROCESSED
#spark.sql("CREATE DATABASE IF NOT EXISTS processed_db LOCATION '/mnt/datalakemlopsd4m/processed/'")

### **Guardamos el df preprocesado en la tabla DELTA (datos_turno1_deltavf) dentro de la BD (processed_db)**

In [0]:
spark_datos.write.format("delta").mode("overwrite").saveAsTable("processed_db.datos_processed_delta")

Verificamos que se creo la Tabla DELTA en la BD adecuada

In [0]:
# Listar todas las bases de datos
#spark.sql("SHOW DATABASES").show()

# Listar las tablas en una base de datos específica (por ejemplo, processed_db)
spark.sql("SHOW TABLES IN processed_db").show()